In [1]:
from Algoritmo_diff_temporal import get_statistics

In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Carregando os dados

In [3]:
window = 1000
path = r"F:/Projetos/apneias_npy"

In [4]:
stats = get_statistics(path, window, n_jobs=-1)

In [5]:
len(stats)

56

In [6]:
stats["Apneia10_0-Luiz Fernando-Intel 5300-Rapida-Sentado_frente-FFZ-_1m-Intel 5300-28_09_2022-10_40-99_92Hz-54R-69bpm.npy"].shape

(2, 57, 11000)

> (Variancia, desvio padrao), (subports), (statistics dos pacotes)

In [7]:
# temos que cada valor foi calculado sobre uma janela de 1000 pacotes.
# isso tem um problema nas transições entre apneia e não apneia...
# uma primeira abordagem é descartar esses valores por enquanto...
# para descartar, devemos pensar que cada valor que temos nesse dataset gerado ->
# foi calculado em cima de 1000 pacotes passados.
# Portanto, o primeiro pacote vale os primeiros 1000 pacotes da captura original.
# Seguindo essa lógica, o pacote de numero 8000 aqui vale do pacote 8000 ao 9000 original.
# A partir do 8001 já começa a faixa de transicao, que só se encerra no pacote 9000, que vale do 9000 ao 10000

In [10]:
# Devemos organizar em apneia e não apneia

def separa_apneia_naoapneia(captura, window_size=1000):
    """
    Separa uma captura em apneia e não apneia. Descartamos a janela de transição (9001 - window:9001 até 9000:...)
    :param captura: np.array de shape (2, 57, windows)
    :param window_size: int
    :return: tuple -> shape=(2, 57, 2, windows_apneia OR windows_naoapneia)
    (apneia e não apneia), (qtd de subports), (variancia e desvio padrao), (qtd de janelas analisadas)
    """
    # para cada subportadora
    subports_apneia = []
    subports_naoapneia = []
    for sub_idx in range(captura.shape[1]):
        nao_apneia = captura[:, sub_idx, :9000-window_size]
        apneia = captura[:, sub_idx, 9000:12000-window_size]
        subports_apneia.append(apneia)
        subports_naoapneia.append(nao_apneia)

    return np.array(subports_apneia), np.array(subports_naoapneia)

In [15]:
# agora, vamos separar para todas as capturas e armazenar em um dicionario
dados_separados = {}
for key in stats.keys():
    dados_separados[key] = separa_apneia_naoapneia(stats[key])


In [19]:
len(dados_separados.items())

56

In [20]:
dados_separados.keys()

dict_keys(['Apneia10_0-Luiz Fernando-Intel 5300-Rapida-Sentado_frente-FFZ-_1m-Intel 5300-28_09_2022-10_40-99_92Hz-54R-69bpm.npy', 'Apneia10_1-Luiz Fernando-Intel 5300-Rapida-Sentado_frente-FFZ-_1m-Intel 5300-28_09_2022-10_43-99_96Hz-56R-71bpm.npy', 'Apneia11_0-Wilson-Intel 5300-Normal-Sentado_ladoDir-FFZ-_1m-Intel 5300-06_10_2022-12_27-99_94Hz-29R-83bpm.npy', 'Apneia11_1-Wilson-Intel 5300-Normal-Sentado_ladoDir-FFZ-_1m-Intel 5300-06_10_2022-12_32-99_94Hz-24R-90bpm.npy', 'Apneia12_0-Wilson-Intel 5300-Normal-Sentado_frente-FFZ-_1m-Intel 5300-06_10_2022-12_38-99_93Hz-28R-96bpm.npy', 'Apneia12_1-Wilson-Intel 5300-Normal-Sentado_frente-FFZ-_1m-Intel 5300-06_10_2022-12_42-99_89Hz-27R-95bpm.npy', 'Apneia13_0-Wilson-Intel 5300-Normal-Sentado_costas-FFZ-_1m-Intel 5300-06_10_2022-12_54-99_93Hz-27R-87bpm.npy', 'Apneia13_1-Wilson-Intel 5300-Normal-Sentado_costas-FFZ-_1m-Intel 5300-06_10_2022-13_01-99_95Hz-24R-86bpm.npy', 'Apneia14_0-Wilson-Intel 5300-Normal-Sentado_ladoEsq-FFZ-_1m-Intel 5300-06_10

> Os dados estão na estrutura adequada. Agora, chegou a hora de limparmos.

## Limpeza e organização de dados para modelos:

> Recapitulando: Temos uma estrutura chamada dados_separados que contém, para cada captura de respiração gravada, a divisão já em apneia e não apneia, para cada uma das 57 subportadoras as informações de variância e desvio padrão que correspondem ao número de janelas analisadas.

In [21]:
# devemos retirar os dados de calibração:
calibracao = []
for key in dados_separados.keys():
    if "calibracao" in key:
        calibracao.append(key)


In [22]:
calibracao

['calibracao-0-Intel 5300-Rapida-Sentado_frente-FFZ-_1m-Intel 5300-22_09_2022-18_54-99_94Hz-0R-0bpm.npy',
 'calibracao-cadeira-cadeira-Intel 5300-Normal-Sentado_frente-FFZ-_1m-Intel 5300-22_09_2022-18_46-99_95Hz-0R-0bpm.npy']

In [23]:
dados_calibracao = [dados_separados[x] for x in calibracao]

In [27]:
dados_separados.pop(calibracao[0])
dados_separados.pop(calibracao[1])

KeyError: 'calibracao-0-Intel 5300-Rapida-Sentado_frente-FFZ-_1m-Intel 5300-22_09_2022-18_54-99_94Hz-0R-0bpm.npy'

In [28]:
len(dados_separados)

54

In [29]:
# agora vamos juntar os dados de apneia e os dados de nao apneia
import pandas as pd

In [ ]:
for key in dados_separados.keys():
    dados_separados[key]